In [ ]:
import os
import sys
import pickle
from pyproj import CRS
import pandas as pd
import geopandas as gpd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import Scenario
from network_wrangler import WranglerLogger
from network_wrangler import ProjectCard

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc
from lasso import util

%load_ext autoreload
%autoreload 2



In [ ]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.setLevel(logging.INFO)

## Remote I/O

In [ ]:
pickle_dir = "../../" #Set to directory where the pickle files are stored
card_dir = "../../ProjectCards"#Set to directory where the cards are stored
transit_dir = "../../Transit"
lasso_dir = '..'
transit_project_card_dir = os.path.join(card_dir, "transit")
highway_project_card_dir = os.path.join(card_dir, '2035','1-2')

parameters = Parameters(lasso_base_dir = lasso_dir)


# Load Scenario

In [ ]:
pickle_file_name = os.path.join(pickle_dir, "scenario_pre_2020.pickle")
curr_scenario = pickle.load(open(pickle_file_name, 'rb'))

# Step 01: Apply Associal Highway Project Cards

In [ ]:
step1_version_04_scenario = Scenario.create_scenario(
    base_scenario = curr_scenario,
    card_directory = highway_project_card_dir,
    project_cards_list=[], 
    validate_project_cards=False
)

step1_version_04_scenario.apply_all_projects()

In [ ]:
step1_version_04_scenario.road_net.links_df = gpd.GeoDataFrame(step1_version_04_scenario.road_net.links_df, geometry=step1_version_04_scenario.road_net.links_df.geometry)
step1_version_04_scenario.road_net.nodes_df = gpd.GeoDataFrame(step1_version_04_scenario.road_net.nodes_df, geometry=step1_version_04_scenario.road_net.nodes_df.geometry)
step1_version_04_scenario.transit_net.set_roadnet(step1_version_04_scenario.road_net, validate_consistency=False)

# Step 02: Apply Transit Project Cards

In [ ]:
step2_version_04_scenario = Scenario.create_scenario(
    base_scenario = step1_version_04_scenario,
    card_directory = transit_project_card_dir,
    project_cards_list=[],
    validate_project_cards=False
)
step2_version_04_scenario.apply_all_projects()


In [ ]:
# in case any transit shapes are broken because of the roadway network changes
step2_version_04_scenario.update_transit_net_with_new_road_net()

# Write model network for Cube

In [ ]:
standard_transit_net = StandardTransit.fromTransitNetwork(step2_version_04_scenario.transit_net, parameters=parameters)
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(transit_dir, "2035", "transit.lin"))